# 데이터프레임 병합

Pandas는 두 개 이상의 데이터프레임을 하나로 합치는 데이터 병합(merge)이나 연결(concatenate)을 지원한다.

## `merge` 명령을 사용한 데이터프레임 병합

`merge` 명령은 두 데이터 프레임의 공통 열 혹은 인덱스를 기준으로 두 개의 테이블을 합친다. 이 때 기준이 되는 열, 행의 데이터를 **키(key)**라고 한다.

In [1]:
import pandas as pd
import numpy as np

df1 = pd.DataFrame({
    '고객번호': [1001, 1002, 1003, 1004, 1005, 1006, 1007],
    '이름': ['남준', '지민', '호석', '석진', '윤기', '정국', '태형']
}, columns=['고객번호', '이름'])
df1

,고객번호,이름
0,1001,남준
1,1002,지민
2,1003,호석
3,1004,석진
4,1005,윤기
5,1006,정국
6,1007,태형


In [2]:
df2 = pd.DataFrame({
    '고객번호': [1001, 1001, 1005, 1006, 1008, 1001],
    '금액': [10000, 20000, 15000, 5000, 100000, 30000]
}, columns=['고객번호', '금액'])
df2

,고객번호,금액
0,1001,10000
1,1001,20000
2,1005,15000
3,1006,5000
4,1008,100000
5,1001,30000


`merge` 명령으로 위의 두 데이터프레임 df1, df2 를 합치면 **공통 열인 `고객번호` 열을 기준으로 데이터를 찾아서 합친다**. 이 때 기본적으로는 양쪽 데이터프레임에 모두 키가 존재하는 데이터만 보여주는 inner join 방식을 사용한다.  

  - default : `inner join 방식` (공통되는 열(고객번호)를 기준으로 left와 light에 모두 존재하는 데이터만 가져옴)

In [3]:
pd.merge(df1, df2)

,고객번호,이름,금액
0,1001,남준,10000
1,1001,남준,20000
2,1001,남준,30000
3,1005,윤기,15000
4,1006,정국,5000


- `outer join 방식` : 키 값이 한쪽에만 있어도 데이터를 보여준다.

In [4]:
pd.merge(df1, df2, how='outer')

,고객번호,이름,금액
0,1001,남준,10000.0
1,1001,남준,20000.0
2,1001,남준,30000.0
3,1002,지민,NaN
4,1003,호석,NaN
5,1004,석진,NaN
6,1005,윤기,15000.0
7,1006,정국,5000.0
8,1007,태형,NaN
9,1008,NaN,100000.0


- `left, right 방식` : 각각 첫번째, 혹은 두번째 데이터프레임의 **키 값을 모두** 보여준다.

In [5]:
pd.merge(df1, df2, how='left')  # 왼쪽 테이블(df1) 기준

,고객번호,이름,금액
0,1001,남준,10000.0
1,1001,남준,20000.0
2,1001,남준,30000.0
3,1002,지민,NaN
4,1003,호석,NaN
5,1004,석진,NaN
6,1005,윤기,15000.0
7,1006,정국,5000.0
8,1007,태형,NaN


In [6]:
pd.merge(df1, df2, how='right')   # 오른쪽 테이블(df2) 기준

,고객번호,이름,금액
0,1001,남준,10000
1,1001,남준,20000
2,1005,윤기,15000
3,1006,정국,5000
4,1008,NaN,100000
5,1001,남준,30000


만약 테이블에 키 값이 같은 데이터가 여러개 있는 경우에는 있을 수 있는 모든 경우의 수를 따져서 조합을 만들어 낸다.
  
- 품종과 꽃잎길이 컬럼이 공통으로 존재하므로 가능한 모든 조합으로 인덱스를 만들어 낸다.

In [7]:
df1 = pd.DataFrame({
    '품종': ['setosa', 'setosa', 'virginica', 'virginica'],
    '꽃잎길이': [1.4, 1.3, 1.5, 1.3]},
    columns=['품종', '꽃잎길이'])
df1

,품종,꽃잎길이
0,setosa,1.4
1,setosa,1.3
2,virginica,1.5
3,virginica,1.3


In [8]:
df2 = pd.DataFrame({
    '품종': ['setosa', 'virginica', 'virginica', 'versicolor'],
    '꽃잎너비': [0.4, 0.3, 0.5, 0.3]},
    columns=['품종', '꽃잎너비'])
df2

,품종,꽃잎너비
0,setosa,0.4
1,virginica,0.3
2,virginica,0.5
3,versicolor,0.3


이 데이터에서 키 값 setosa에 대해 왼쪽 데이터프레임는 1.4와 1.3라는 2개의 데이터, 오른쪽 데이터프레임에 0.4라는 1개의 데이터가 있으므로 병합된 데이터에는 setosa가 (1.4, 0.4), (1.3, 0.4) 두 개의 데이터가 생긴다. 키 값 virginica의 경우에는 왼쪽 데이터프레임에 1.5와 1.3라는 2개의 데이터, 오른쪽 데이터프레임에 0.3와 0.5라는 2개의 데이터가 있으므로 2개와 2개의 조합에 의해 4가지 값이 생긴다.

In [9]:
pd.merge(df1, df2)  # 모든 조합으로 데이터 발생

,품종,꽃잎길이,꽃잎너비
0,setosa,1.4,0.4
1,setosa,1.3,0.4
2,virginica,1.5,0.3
3,virginica,1.5,0.5
4,virginica,1.3,0.3
5,virginica,1.3,0.5


* 두 데이터프레임에서 **이름이 같은 열은 모두 키가 된다.**   


* 만약 이름이 같아도 키가 되면 안되는 열이 있다면 `on` 인수로 기준열을 명시해야 한다. 

  다음 예에서 첫번째 데이터프레임의 "데이터"는 실제로는 금액을 나타내는 데이터이고 두번째 데이터프레임의 "데이터"는 실제로는 성별을 나타내는 데이터이므로 **이름이 같아도 다른 데이터**이다. 따라서 이 열은 기준열이 되면 안된다.

In [10]:
df1 = pd.DataFrame({
    '고객명': ['채영', '채영', '정국'],
    '날짜': ['2021-01-01', '2021-01-02', '2021-01-01'],
    '데이터': ['20000', '30000', '100000']})
df1

,고객명,날짜,데이터
0,채영,2021-01-01,20000
1,채영,2021-01-02,30000
2,정국,2021-01-01,100000


In [11]:
df2 = pd.DataFrame({
    '고객명': ['채영', '정국'],
    '데이터': ['여자', '남자']})
df2

,고객명,데이터
0,채영,여자
1,정국,남자


In [12]:
pd.merge(df1, df2, on='고객명')

,고객명,날짜,데이터_x,데이터_y
0,채영,2021-01-01,20000,여자
1,채영,2021-01-02,30000,여자
2,정국,2021-01-01,100000,남자


이 때 기준 열이 아니면서 이름이 같은 열에는 `_x` 또는 `_y` 와 같은 접미사가 붙는다.

* 반대로 **키가 되는 기준열의 이름이 두 데이터프레임에서 다르다면** `left_on`, `right_on` 인수를 사용하여 **기준열을 명시**해야 한다.

In [13]:
df1 = pd.DataFrame({
    '이름': ['제니', '태형', '태형'],
    '성적': [1, 2, 3]})
df1

,이름,성적
0,제니,1
1,태형,2
2,태형,3


In [14]:
df2 = pd.DataFrame({
    '성명': ['제니', '제니', '태형'],
    '성적2': [4, 5, 6]})
df2

,성명,성적2
0,제니,4
1,제니,5
2,태형,6


df1(왼쪽)과 df2(오른쪽) 데이터프레임에서 각각 "이름"과 "성명"으로 컬럼명이 다르므로, 아래와 같이 left_on, right_on을 지정하여 두 열이 같은 열임을 명기한다.

In [15]:
pd.merge(df1, df2, left_on='이름', right_on="성명")  # 이름과 성명 컬럼이 같은 컬럼임을 명기

,이름,성적,성명,성적2
0,제니,1,제니,4
1,제니,1,제니,5
2,태형,2,태형,6
3,태형,3,태형,6


In [16]:
pd.merge(df1, df2)  # Inner Join 방식에서 겹치는 행이 없으므로 데이터가 하나도 안나옴

MergeError: No common columns to perform merge on. Merge options: left_on=None, right_on=None, left_index=False, right_index=False

일반 데이터 열이 아닌 인덱스를 기준열로 사용하려면 `left_index` 또는 `right_index` 인수를 `True` 로 설정한다.

In [17]:
df1 = pd.DataFrame({
    '도시': ['서울', '서울', '서울', '부산', '부산'],
    '연도': [2000, 2005, 2010, 2000, 2005],
    '인구': [9853972, 9762546, 9631482, 3655437, 3512547]})
df1

,도시,연도,인구
0,서울,2000,9853972
1,서울,2005,9762546
2,서울,2010,9631482
3,부산,2000,3655437
4,부산,2005,3512547


In [18]:
df2 = pd.DataFrame(
    np.arange(12).reshape((6, 2)),
    index=[['부산', '부산', '서울', '서울', '서울', '서울'],
           [2000, 2005, 2000, 2005, 2010, 2015]],
    columns=['데이터1', '데이터2'])
df2

데이터1  데이터2
부산 2000     0     1
   2005     2     3
서울 2000     4     5
   2005     6     7
   2010     8     9
   2015    10    11

* 우측 데이터 프레임에는 인덱스가 2개 있고, 해당하는 행이 좌측에는 `['도시', '연도']`임을 명기하여 데이터를 inner join 시킴

In [19]:
pd.merge(df1, df2, left_on=['도시', '연도'], right_index=True)

,도시,연도,인구,데이터1,데이터2
0,서울,2000,9853972,4,5
1,서울,2005,9762546,6,7
2,서울,2010,9631482,8,9
3,부산,2000,3655437,0,1
4,부산,2005,3512547,2,3


In [20]:
df1 = pd.DataFrame(
    [[1., 2.], [3., 4.], [5., 6.]],
    index=['a', 'c', 'e'],
    columns=['서울', '부산'])
df1

,서울,부산
a,1.0,2.0
c,3.0,4.0
e,5.0,6.0


In [21]:
df2 = pd.DataFrame(
    [[7., 8.], [9., 10.], [11., 12.], [13, 14]],
    index=['b', 'c', 'd', 'e'],
    columns=['대구', '광주'])
df2

,대구,광주
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,13.0,14.0


In [22]:
pd.merge(df1, df2, how='outer', left_index=True, right_index=True)

,서울,부산,대구,광주
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


### join 메서드

`merge` 명령어 대신 `join` 메서드를 사용할 수도 있다.

In [23]:
df1.join(df2, how='outer')  # pd.merge(df1, df2)와 달리 df1.join(df2)로 문법이 다르다.

,서울,부산,대구,광주
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


#### 연습 문제 4.2.10

두 개의 데이터프레임을 만들고 merge 명령으로 합친다. 단 데이터프레임은 다음 조건을 만족해야 한다.

1. 각각 5 x 5 이상의 크기를 가진다.
1. 공통 열을 하나 이상 가진다. 다만 공통 열의 이름은 서로 다르다.


In [24]:
data_dict = {'종목':  [11111, 22222, 33333, 44444, 55555],
             'open':  [11650, 11100, 11200, 11100, 11000],
             'high':  [12100, 11800, 11200, 11100, 11150],
             'low' :  [11600, 11050, 10900, 10950, 10900],
             'close': [11900, 11600, 11000, 11100, 11050]}

df3 = pd.DataFrame(data_dict)
df3

,종목,open,high,low,close
0,11111,11650,12100,11600,11900
1,22222,11100,11800,11050,11600
2,33333,11200,11200,10900,11000
3,44444,11100,11100,10950,11100
4,55555,11000,11150,10900,11050


In [25]:
data_dict = {'코드':  [11111, 33333, 55555, 77777],
             '회사명':  ['ABC', 'DEF', 'GHI', 'JKH']}

df4 = pd.DataFrame(data_dict)
df4

,코드,회사명
0,11111,ABC
1,33333,DEF
2,55555,GHI
3,77777,JKH


In [26]:
merged_df = pd.merge(df3, df4, left_on='종목', right_on='코드', 
                  how='inner')
#                   how='outer')
#                   how='right')
#                   how='left')
merged_df

,종목,open,high,low,close,코드,회사명
0,11111,11650,12100,11600,11900,11111,ABC
1,33333,11200,11200,10900,11000,33333,DEF
2,55555,11000,11150,10900,11050,55555,GHI


In [27]:
del merged_df['코드']
merged_df

,종목,open,high,low,close,회사명
0,11111,11650,12100,11600,11900,ABC
1,33333,11200,11200,10900,11000,DEF
2,55555,11000,11150,10900,11050,GHI


## `concat` 명령을 사용한 데이터 연결

`concat` 명령을 사용하면 기준 열(key column)을 사용하지 않고 단순히 데이터를 연결(concatenate)한다.

기본적으로는 위/아래로 데이터 행을 연결한다. 단순히 두 시리즈나 데이터프레임을 연결하기 때문에 인덱스 값이 중복될 수 있다. 

In [28]:
s1 = pd.Series([0, 1], index=['A', 'B'])
s2 = pd.Series([2, 3, 4], index=['A', 'B', 'C'])

In [29]:
s1

A    0
B    1
dtype: int64

In [30]:
s2

A    2
B    3
C    4
dtype: int64

In [31]:
pd.concat([s1, s2])

A    0
B    1
A    2
B    3
C    4
dtype: int64

만약 옆으로 데이터 열을 연결하고 싶으면 `axis=1`로 인수를 설정한다.

In [32]:
df1 = pd.DataFrame(
    np.arange(6).reshape(3, 2),
    index=['a', 'b', 'c'],
    columns=['데이터1', '데이터2'])
df1

,데이터1,데이터2
a,0,1
b,2,3
c,4,5


In [33]:
df2 = pd.DataFrame(
    5 + np.arange(4).reshape(2, 2),
    index=['a', 'c'],
    columns=['데이터3', '데이터4'])
df2

,데이터3,데이터4
a,5,6
c,7,8


In [34]:
pd.concat([df1, df2], axis=1)

,데이터1,데이터2,데이터3,데이터4
a,0,1,5.0,6.0
b,2,3,NaN,NaN
c,4,5,7.0,8.0


#### 연습 문제 4.2.11

어느 회사의 전반기(1월 ~ 6월) 실적을 나타내는 데이터프레임과 후반기(7월 ~ 12월) 실적을 나타내는 데이터프레임을 만든 뒤 합친다.
실적 정보는 "매출", "비용", "이익" 으로 이루어진다. (이익 = 매출 - 비용).

또한 1년간의 총 실적을 마지막 행으로 덧붙인다.

In [40]:
df1 = pd.DataFrame({
    '매출':[1000,1500,3000,4000,5000,6000],
    '비용':[1500,2000,2500,2700,3000,3200]},
    index = ['1월', '2월', '3월', '4월', '5월' ,'6월'],
    columns = ['매출', '비용']
)

df2 = pd.DataFrame({
    '매출':[3000,4500,5000,6000,2000,1000],
    '비용':[2500,2000,1500,2700,1000,1200]},
    index = ['7월', '8월', '9월', '10월', '11월' ,'12월'],
    columns = ['매출', '비용']
)

display(df1)
display(df2)

,매출,비용
1월,1000,1500
2월,1500,2000
3월,3000,2500
4월,4000,2700
5월,5000,3000
6월,6000,3200


,매출,비용
7월,3000,2500
8월,4500,2000
9월,5000,1500
10월,6000,2700
11월,2000,1000
12월,1000,1200


In [41]:
df3 = pd.concat([df1, df2]) # , axis=0)
df3

,매출,비용
1월,1000,1500
2월,1500,2000
3월,3000,2500
4월,4000,2700
5월,5000,3000
6월,6000,3200
7월,3000,2500
8월,4500,2000
9월,5000,1500
10월,6000,2700


In [42]:
df3["이익"] = df3["매출"] - df3["비용"]
df3

,매출,비용,이익
1월,1000,1500,-500
2월,1500,2000,-500
3월,3000,2500,500
4월,4000,2700,1300
5월,5000,3000,2000
6월,6000,3200,2800
7월,3000,2500,500
8월,4500,2000,2500
9월,5000,1500,3500
10월,6000,2700,3300


In [43]:
df3.loc["총실적"] = df3.sum(axis=0)
df3

,매출,비용,이익
1월,1000,1500,-500
2월,1500,2000,-500
3월,3000,2500,500
4월,4000,2700,1300
5월,5000,3000,2000
6월,6000,3200,2800
7월,3000,2500,500
8월,4500,2000,2500
9월,5000,1500,3500
10월,6000,2700,3300
